In [ ]:
!git clone https://github.com/yaq007/Autofocus-Layer.git
!wget http://cseweb.ucsd.edu/~yaq007/models.tar.gz
!tar -xzvf models.tar.gz
!rm models.tar.gz
!wget http://cseweb.ucsd.edu/~yaq007/dataset.zip
!unzip dataset.zip
!rm dataset.zip
!pip install nibabel
!pip install SimpleITK
!cp -r ./Autofocus-Layer/* ./

In [ ]:
import imageio
from IPython.display import Image
import numpy as np
import SimpleITK as sitk 
import nibabel as nib

In [ ]:
!python test.py --num_gpus 1 --id AFN1 --test_epoch 390  --visualize

In [ ]:
def norm(img):
    img-=img.min()
    return img/img.max()

In [ ]:
pred = sitk.ReadImage("result/AFN1/VSD.brats_tcia_pat111_0001.42287.mha")
nda = sitk.GetArrayFromImage(pred).transpose(0,2,1)
p = 'HGG/brats_tcia_pat111_0001/VSD.Brain.XX.O.MR_Flair.40827/VSD.Brain.XX.O.MR_Flair.40827.nii.gz'
image = nib.load(p).get_data().transpose(2,0,1)
image = np.expand_dims(image,-1)
image = norm(np.concatenate([image,image,image],axis=-1))
mask = image.copy()
for row in range(1,nda.max()):
    nda2 = nda.copy()
    nda2[nda2!=row] = 0
    nda2*=1
    if row <3:
        mask[:,:,:,row] +=nda2/4.0
    else:
        mask[:,:,:,0] +=nda2/4.0
        mask[:,:,:,1] +=nda2/4.0
nda = np.expand_dims(nda,-1)
nda = norm(np.concatenate([nda,nda,nda],axis=-1))
imageio.mimsave("/tmp/gif.gif", np.concatenate([image,mask,norm(nda)],axis=-2), duration=0.0001)
Image(filename="/tmp/gif.gif", format='png')